# These are experiments for
- django using Parse Trees
- django using BPE

The experimental setup is: using the same amount of data and the same model configuration what are the performance differences between the parse-tree tokenizer and a regularr BPE tokenizer

In [1]:
from src.dataset_loaders import CoNaLa_RawDataLoader, Django_RawDataLoader, Parseable_Django_RawDataLoader
from src.DataProcessors import Parse_Tree_Translation_DataProcessor, CodeTrainedBPE_Translation_DataProcessor
from src.translation_transformer import Translation_Transformer
from src.Experiments import TranslationExperiment
from src.tree_sitter_AST_utils import Node_Processor
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [9]:
%%capture
filtered_django_raw_data_loader = Parseable_Django_RawDataLoader(data_directory="/my_shared/notebooks/datasets/django/", 
                                                                 tokenizer_dir="datasets/code_search_net/",
                                                                 grammar_path="src/external_repos/tree-sitter-python/src/grammar.json",
                                                                 parser_library_path='src/external_repos/tree-sitter-python')
train_pairs = filtered_django_raw_data_loader.english_to_code_for_translation("train")

Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Could not parse and reconstruct: if DEFAULT_CACHE_ALIAS not in settings . CACHES :
Could not parse and reconstruct: raise ImproperlyConfigured ( "You must define a '%s' cache" % DEFAULT_CACHE_ALIAS )
Could not parse and reconstruct: def get_cache ( backend , ** kwargs ) :
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Could not parse and reconstruct: def _create_cache ( backend , ** kwargs ) :
Could not parse and reconstruct: try :
Could not parse and reconstruct: try :
Tree generation terminated
Could not parse and reconstruct: except KeyError :
Could not parse and reconstruct: try :
Tree generation terminated
Could not parse and reconstruct: except ImportErro

In [11]:
len(train_pairs)

8446

## Pairs are now filtered to those that can be seen by the model
## Training with Parse Trees

In [16]:
parse_tree_processor = Parse_Tree_Translation_DataProcessor(train_pairs, tokenizer_dir="datasets/code_search_net/",
                                                            grammar_path="src/external_repos/tree-sitter-python/src/grammar.json",
                                                            parser_library_path='src/external_repos/tree-sitter-python')

Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
T

In [17]:
train_dataloader = parse_tree_processor.to_dataloader(16, num_workers=0)

In [18]:
translator = Translation_Transformer(parse_tree_processor,
                                     parse_tree_processor.vocab_size, 
                                     embed_dim=256, 
                                     att_heads=4, 
                                     layers=4, 
                                     dim_feedforward=1024, 
                                     max_seq_length=550,
                                     use_copy=False)
translator.model.to("cuda")
translator.model.device

device(type='cuda', index=0)

In [19]:
translator.load("translator_Django_Parse_tree.pytorch")

FileNotFoundError: [Errno 2] No such file or directory: 'translator_Django_Parse_tree.pytorch'

In [20]:
translator.train(100, train_dataloader)

Keyboard Interrupt!


## Training with BPE

In [5]:
BPE_processor = CodeTrainedBPE_Translation_DataProcessor(train_pairs)

In [7]:
train_dataloader = parse_tree_processor.to_dataloader(32, num_workers=0)

In [6]:
translator = Translation_Transformer(BPE_processor,
                                     BPE_processor.vocab_size, 
                                     embed_dim=256, 
                                     att_heads=4, 
                                     layers=4, 
                                     dim_feedforward=1024, 
                                     max_seq_length=550,
                                     use_copy=False)
translator.model.to("cuda")
translator.model.device

device(type='cuda', index=0)

In [7]:
translator.load("./translator_code_BPE_django.pytorch")

In [ ]:
translator.train(100, train_dataloader)

# Evaluating

In [21]:
translator.raw_batch_predict(["x was a string `a`", "foo is a list"])

Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated
Tree generation terminated


[('x = "x"\n', <src.tree_sitter_AST_utils.PartialTree at 0x7fc83bc42b00>),
 ('foo = foo()\n', <src.tree_sitter_AST_utils.PartialTree at 0x7fc83c10fb70>)]

In [38]:
def translation_batch_predictor(src_sents):
    outputs = tranX_translator.raw_batch_predict(src_sents)
    return [code for code in outputs]

In [39]:
test_pairs = filtered_django_raw_data_loader.english_to_code_for_translation("test")

In [40]:
Django_experiment = TranslationExperiment(test_pairs)

In [42]:
Django_experiment.evaluate(translation_batch_predictor, batched=32, save_dir="./", save_name="translation_eval_tranX_django.txt")


saving translation eval to file: ./translation_eval_tranX_django.txt
| EVALUATION | BLEU: 0.797 |


{'BLEU': 0.797131812764566}

In [31]:
from src.models.tranX_translator import TranX_Prertrained_Translator

tranX_translator = TranX_Prertrained_Translator(dataset="django")

load parser from [src/external_repos/tranX/data/pretrained_models/django.bin]


In [48]:
tranX_translator.raw_predict("_func_defaults is a string __defaults__.")

"_func_defaults = '__defaults__'"